<a href="https://colab.research.google.com/github/archanags001/coding_challenge/blob/main/recommendation_Pearson_correlation_and_Cosine_similarity__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Recommendation System**

#  **STEP 1: Problem Statement** 

Using the sample of masked transactional data, create a recommendation engine consisting of a predictive model & post-processing code that addresses business rules.

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics.pairwise import cosine_similarity

#  **STEP 2: Read and gather knowledge about the data**

In [ ]:
# Reading the data from .txt file using pandas
df = pd.read_csv("",delimiter = "\|")
# Making a copy of the dataframe
df_copy = df.copy()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
# Shape command will give number of samples and number of features in dataset(rows,columns)
df.shape

(41295, 5)

In [ ]:
# Print the first 5 rows of the dataframe.
df.head()

,tid,date,daypart,channel,items
0,1,2016-03-02,A,1,"21d683,21d683"
1,2,2016-03-02,A,1,"aa254f,524e24,2f4a70,6ca4a6,221380,b8bfe8,2213..."
2,3,2016-03-02,A,2,"fe5db1,aa254f,550e0e,7356a8,7356a8"
3,4,2016-03-02,A,2,4d51b8
4,5,2016-03-02,A,2,6050b0


In [ ]:
#info method provides information about dataset like total values in each column, null, not null, datatype, memory occupied etc
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41295 entries, 0 to 41294
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   tid      41295 non-null  int64 
 1   date     41295 non-null  object
 2   daypart  41295 non-null  object
 3   channel  41295 non-null  int64 
 4   items    41295 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.6+ MB


In [ ]:
#Describe gives statistical information about numerical columns in the dataset also we can check any missing vales in columns,
df.describe()

,tid,channel
count,41295.00000,41295.000000
mean,20648.00000,2.087565
std,11920.98402,0.322295
min,1.00000,1.000000
25%,10324.50000,2.000000
50%,20648.00000,2.000000
75%,30971.50000,2.000000
max,41295.00000,3.000000


In [ ]:
# "daypart" distribution
df["daypart"].value_counts()

A    11161
B     9813
E     8394
F     4471
C     4188
D     3268
Name: daypart, dtype: int64

In [ ]:
# channel distribution
df["channel"].value_counts()

2    36689
3     4111
1      495
Name: channel, dtype: int64

# **STEP 3: Clean, Prepare the Data**

In [ ]:
# No of items in the Customer basket
count = df['items'].str.split(',').apply(len)
df["No_of_items"]=count
df.head()

,tid,date,daypart,channel,items,No_of_items
0,1,2016-03-02,A,1,"21d683,21d683",2
1,2,2016-03-02,A,1,"aa254f,524e24,2f4a70,6ca4a6,221380,b8bfe8,2213...",11
2,3,2016-03-02,A,2,"fe5db1,aa254f,550e0e,7356a8,7356a8",5
3,4,2016-03-02,A,2,4d51b8,1
4,5,2016-03-02,A,2,6050b0,1


In [ ]:
# Removing data if no of items in the customer basket less than or equal to 1
df = df[df["No_of_items"] > 1]

In [ ]:
# Find the first item in the customer basket
df['first_item']=df["items"].str.split(",").str.get(0)
df.head()

,tid,date,daypart,channel,items,No_of_items,first_item
0,1,2016-03-02,A,1,"21d683,21d683",2,21d683
1,2,2016-03-02,A,1,"aa254f,524e24,2f4a70,6ca4a6,221380,b8bfe8,2213...",11,aa254f
2,3,2016-03-02,A,2,"fe5db1,aa254f,550e0e,7356a8,7356a8",5,fe5db1
5,6,2016-03-02,A,2,"6050b0,dc9f6a,dc9f6a",3,6050b0
6,7,2016-03-02,A,2,"6050b0,dc9f6a,321958",3,6050b0


In [ ]:
# First item distribution
df['first_item'].value_counts()

31299d    2734
c5f033    2672
7356a8    2650
ea300a    1261
292491    1079
          ... 
f9c182       3
172d33       2
73e6b6       2
e14777       1
946878       1
Name: first_item, Length: 110, dtype: int64

In [ ]:
# Removing rows if only one customer purchase the item as first item
df_item = df[df.groupby('first_item').first_item.transform('count')>1].copy() 
df_item.head()

,tid,date,daypart,channel,items,No_of_items,first_item
0,1,2016-03-02,A,1,"21d683,21d683",2,21d683
1,2,2016-03-02,A,1,"aa254f,524e24,2f4a70,6ca4a6,221380,b8bfe8,2213...",11,aa254f
2,3,2016-03-02,A,2,"fe5db1,aa254f,550e0e,7356a8,7356a8",5,fe5db1
5,6,2016-03-02,A,2,"6050b0,dc9f6a,dc9f6a",3,6050b0
6,7,2016-03-02,A,2,"6050b0,dc9f6a,321958",3,6050b0


In [ ]:
# # Removing rows if only one customer purchase the item as first item
# df_item = df[df.groupby('first_item').first_item.transform('count')>10].copy() 
# df_item.head()

In [ ]:
# df_item['first_item'].value_counts()

In [ ]:
# Reset index
df_item.reset_index(drop=True, inplace=True)
df_item.head()

,tid,date,daypart,channel,items,No_of_items,first_item
0,1,2016-03-02,A,1,"21d683,21d683",2,21d683
1,2,2016-03-02,A,1,"aa254f,524e24,2f4a70,6ca4a6,221380,b8bfe8,2213...",11,aa254f
2,3,2016-03-02,A,2,"fe5db1,aa254f,550e0e,7356a8,7356a8",5,fe5db1
3,6,2016-03-02,A,2,"6050b0,dc9f6a,dc9f6a",3,6050b0
4,7,2016-03-02,A,2,"6050b0,dc9f6a,321958",3,6050b0


##  spliting the dataset to train dataset and test dataset




In [ ]:
# spliting the dataset to train dataset and test dataset
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df_item, df_item["first_item"]):
    train_set = df_item.loc[train_index]
    test_set = df_item.loc[test_index]
print(train_set.shape,test_set.shape )

(25234, 7) (6309, 7)


In [ ]:
# Training set data distribution
train_set['first_item'].value_counts()/ len(train_set)

31299d    0.086669
c5f033    0.084687
7356a8    0.084014
ea300a    0.039986
292491    0.034200
            ...   
f9c182    0.000079
172d33    0.000079
20b01a    0.000079
73e6b6    0.000079
e13c5c    0.000079
Name: first_item, Length: 108, dtype: float64

In [ ]:
# Test set data distribution
test_set['first_item'].value_counts()/ len(test_set)

31299d    0.086702
c5f033    0.084799
7356a8    0.084007
ea300a    0.039943
292491    0.034237
            ...   
f9c182    0.000159
cb06d4    0.000159
7a0017    0.000159
86100a    0.000159
20b01a    0.000159
Name: first_item, Length: 106, dtype: float64

In [ ]:
# Make a copy of training set
df = train_set
df.head()

,tid,date,daypart,channel,items,No_of_items,first_item
28164,36746,2016-03-29,C,2,"e051ee,ea300a",2,e051ee
31479,41202,2016-04-01,F,2,"230526,330e2e,7a0017,9fc08c",4,230526
26311,34338,2016-03-27,E,2,"4c85c6,b59cb7",2,4c85c6
8134,10478,2016-03-10,A,2,"1bfcc7,7356a8,363929,363929,dc9f6a",5,1bfcc7
5192,6649,2016-03-07,A,2,"1bfcc7,1bfcc7,1bfcc7,1bfcc7,1bfcc7,1bfcc7,7a00...",8,1bfcc7


In [ ]:
# Quantity of item in customer basket
def item_count(str):
    counts = dict()
    words = str.split(",")

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

item_quantity =[]
for i in df['items']:
  item_quantity.append(item_count(i))
df["item_quantity"] = item_quantity
df['item_quantity'] = df['item_quantity'].astype(str)
df.head()

,tid,date,daypart,channel,items,No_of_items,first_item,item_quantity
28164,36746,2016-03-29,C,2,"e051ee,ea300a",2,e051ee,"{'e051ee': 1, 'ea300a': 1}"
31479,41202,2016-04-01,F,2,"230526,330e2e,7a0017,9fc08c",4,230526,"{'230526': 1, '330e2e': 1, '7a0017': 1, '9fc08..."
26311,34338,2016-03-27,E,2,"4c85c6,b59cb7",2,4c85c6,"{'4c85c6': 1, 'b59cb7': 1}"
8134,10478,2016-03-10,A,2,"1bfcc7,7356a8,363929,363929,dc9f6a",5,1bfcc7,"{'1bfcc7': 1, '7356a8': 1, '363929': 2, 'dc9f6..."
5192,6649,2016-03-07,A,2,"1bfcc7,1bfcc7,1bfcc7,1bfcc7,1bfcc7,1bfcc7,7a00...",8,1bfcc7,"{'1bfcc7': 6, '7a0017': 1, 'ea300a': 1}"


In [ ]:
# Drop "items"
df.drop(["items"],axis = 1,inplace=True)

# Transform each item of a list-like to a row, replicating index values.
df_new = df.assign(sep_item=df.item_quantity.str.split(",")).explode('sep_item')
df_new.head()

,tid,date,daypart,channel,No_of_items,first_item,item_quantity,sep_item
28164,36746,2016-03-29,C,2,2,e051ee,"{'e051ee': 1, 'ea300a': 1}",{'e051ee': 1
28164,36746,2016-03-29,C,2,2,e051ee,"{'e051ee': 1, 'ea300a': 1}",'ea300a': 1}
31479,41202,2016-04-01,F,2,4,230526,"{'230526': 1, '330e2e': 1, '7a0017': 1, '9fc08...",{'230526': 1
31479,41202,2016-04-01,F,2,4,230526,"{'230526': 1, '330e2e': 1, '7a0017': 1, '9fc08...",'330e2e': 1
31479,41202,2016-04-01,F,2,4,230526,"{'230526': 1, '330e2e': 1, '7a0017': 1, '9fc08...",'7a0017': 1


In [ ]:
# Drop 'item_quantity'
df_new.drop(['item_quantity'],axis=1, inplace=True)

# Cleaning the item data
df_new['Item'] =df_new['sep_item'].str.replace('{',"")
df_new['Item'] =df_new['Item'].str.replace('}',"")
df_new['Item'] =df_new['Item'].str.replace("'","")
df_new.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,tid,date,daypart,channel,No_of_items,first_item,sep_item,Item
28164,36746,2016-03-29,C,2,2,e051ee,{'e051ee': 1,e051ee: 1
28164,36746,2016-03-29,C,2,2,e051ee,'ea300a': 1},ea300a: 1
31479,41202,2016-04-01,F,2,4,230526,{'230526': 1,230526: 1
31479,41202,2016-04-01,F,2,4,230526,'330e2e': 1,330e2e: 1
31479,41202,2016-04-01,F,2,4,230526,'7a0017': 1,7a0017: 1


In [ ]:
# Seperate item and quantity
df_new["Items"] = df_new["Item"].str.split(":").str.get(0)
df_new["Quantity"] = df_new["Item"].str.split(":").str.get(1)
df_new.head()

,tid,date,daypart,channel,No_of_items,first_item,sep_item,Item,Items,Quantity
28164,36746,2016-03-29,C,2,2,e051ee,{'e051ee': 1,e051ee: 1,e051ee,1
28164,36746,2016-03-29,C,2,2,e051ee,'ea300a': 1},ea300a: 1,ea300a,1
31479,41202,2016-04-01,F,2,4,230526,{'230526': 1,230526: 1,230526,1
31479,41202,2016-04-01,F,2,4,230526,'330e2e': 1,330e2e: 1,330e2e,1
31479,41202,2016-04-01,F,2,4,230526,'7a0017': 1,7a0017: 1,7a0017,1


In [ ]:
# removing extra space in the item
item_list = []
for i in df_new['Items']:
 item_list.append(i.strip())
df_new["Items"] = item_list
df_new["Items"].value_counts()

ea300a    5097
c5f033    4952
31299d    4661
e051ee    3598
7356a8    3420
          ... 
172d33       6
f9c182       6
3ef6c2       6
20b01a       5
946878       2
Name: Items, Length: 113, dtype: int64

In [ ]:
# # Removing rows if only one customer purchase the item as first item
# df_new = df[df.groupby("Items").first_item.transform('count')>10].copy() 
# df_new.head()

In [ ]:
df_new.drop(['date','sep_item','Item','No_of_items','first_item' ],axis=1, inplace=True)
df_new.head()

,tid,daypart,channel,Items,Quantity
28164,36746,C,2,e051ee,1
28164,36746,C,2,ea300a,1
31479,41202,F,2,230526,1
31479,41202,F,2,330e2e,1
31479,41202,F,2,7a0017,1


In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75141 entries, 28164 to 8331
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tid       75141 non-null  int64 
 1   daypart   75141 non-null  object
 2   channel   75141 non-null  int64 
 3   Items     75141 non-null  object
 4   Quantity  75141 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [ ]:
df_new['Quantity'] = df_new['Quantity'].astype(int)

##  Prepare data for Recommendation System

In [ ]:
# This states the quantity of units of each item present in each customer’s basket.
Customer_Item_matrix = pd.pivot_table(df_new, index=['channel','daypart','tid'], columns='Items', values='Quantity',aggfunc='sum').fillna(0)
Customer_Item_matrix.head(10)

Items                 03bdf7  086cbd  0c816e  11170d  11c5cc  15d7d5  172d33  \
channel daypart tid                                                            
1       A       2        0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                1289     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                2541     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                2542     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                2543     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                2545     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                2546     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                2547     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                3823     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
                5169     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

Items                 192a28  1bfcc7  20b01a  ...  e93ce9  ea300a  ea32e3  \
channel daypart tid                           ...                           
1       A       2        0.0     0.0     0.0  ...     0.0     0.0     0.0   
                1289     0.0     0.0     0.0  ...     0.0     1.0     0.0   
                2541     0.0     0.0     0.0  ...     0.0     0.0     0.0   
                2542     0.0     0.0     0.0  ...     0.0     1.0     0.0   
                2543     0.0     2.0     0.0  ...     0.0     0.0     0.0   
                2545     0.0     2.0     0.0  ...     0.0     0.0     0.0   
                2546     0.0     0.0     0.0  ...     0.0     0.0     0.0   
                2547     0.0     0.0     0.0  ...     0.0     0.0     0.0   
                3823     0.0     0.0     0.0  ...     0.0     0.0     0.0   
                5169     0.0     0.0     0.0  ...     0.0     1.0     0.0   

Items                 eb3af5  f717d0  f9c182  f9ec21  fb1718  fde66d  fe5db1  
channel daypart tid                                                           
1       A       2        0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                1289     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                2541     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                2542     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                2543     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                2545     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                2546     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                2547     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                3823     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
                5169     0.0     0.0     0.0     0.0     0.0     0.0     0.0  

[10 rows x 113 columns]

In [ ]:
Customer_Item_matrix.shape

(25234, 113)

# **STEP 4: Recommendation System**

## Pearson correlation 

In [ ]:
# recommendations system using pearson
def recommendations_pearson(df, item):
    recommendations = df.corrwith(df[item])
    recommendations.dropna(inplace=True)
    recommendations = pd.DataFrame(recommendations, columns=['correlation']).reset_index()
    recommendations = recommendations.sort_values(by='correlation', ascending=False)
    
    return recommendations.iloc[1:4]

In [ ]:
# Random test
recommendations = recommendations_pearson(Customer_Item_matrix, "7356a8")
recommendations.head()

,Items,correlation
28,363929,0.430336
52,8319aa,0.149692
8,1bfcc7,0.111421


### Evaluation

In [ ]:
# make a copy of test set
evaluation_set = test_set.copy()

In [ ]:
# recommendation 
Recommendation =[]
for i in test_set['first_item']:
  recommendations_1 = recommendations_pearson(Customer_Item_matrix, i)
  Recommendation.append(list(recommendations_1['Items']))
evaluation_set["Recommendation"] = Recommendation
evaluation_set.head()

,tid,date,daypart,channel,items,No_of_items,first_item,Recommendation
6423,8235,2016-03-08,A,3,"7356a8,363929",2,7356a8,"[363929, 8319aa, 1bfcc7]"
11685,15147,2016-03-13,B,3,"31299d,c5f033,ea300a",3,31299d,"[e051ee, c5f033, ea300a]"
14209,18375,2016-03-16,A,2,"7356a8,7356a8,363929,1bfcc7",4,7356a8,"[363929, 8319aa, 1bfcc7]"
4895,6269,2016-03-06,E,2,"31299d,31299d,31299d,c5f033,c5f033,ea300a,ea300a",7,31299d,"[e051ee, c5f033, ea300a]"
15853,20582,2016-03-17,E,2,"c5f033,c5f033,ea32e3,70aa12,ea300a",5,c5f033,"[e051ee, 31299d, bd59c4]"


In [ ]:
# method for evaluation
def evaluation(evaluation_set):
  hit = []
  recommnd_list=[]
  total = 0
  for index, row in evaluation_set.iterrows():
    recommnd = row['Recommendation']
    i = row['items']
    total +=1
    if recommnd[0] in i:
      hit.append("True")
      recommnd_list.append(recommnd[0])
      continue
    elif recommnd[1] in i:
      hit.append("True")
      recommnd_list.append(recommnd[1])
      continue
    elif recommnd[2] in i:
      hit.append("True")
      recommnd_list.append(recommnd[2])
      continue
    else:
      hit.append('False')
      recommnd_list.append("0") 
  evaluation_set["Hit"] = hit
  evaluation_set["test"] = recommnd_list
  print(total)
  print(evaluation_set["Hit"].value_counts())
# print(evaluation_set['test'].value_counts())
  return evaluation_set

In [ ]:
# Recommendation system result
result = evaluation(evaluation_set)
result.head()
evaluation_set[['first_item','Recommendation','items','Hit']]

6309
False    4019
True     2290
Name: Hit, dtype: int64


,first_item,Recommendation,items,Hit
6423,7356a8,"[363929, 8319aa, 1bfcc7]","7356a8,363929",True
11685,31299d,"[e051ee, c5f033, ea300a]","31299d,c5f033,ea300a",True
14209,7356a8,"[363929, 8319aa, 1bfcc7]","7356a8,7356a8,363929,1bfcc7",True
4895,31299d,"[e051ee, c5f033, ea300a]","31299d,31299d,31299d,c5f033,c5f033,ea300a,ea300a",True
15853,c5f033,"[e051ee, 31299d, bd59c4]","c5f033,c5f033,ea32e3,70aa12,ea300a",False
...,...,...,...,...
30326,667497,"[e051ee, bf0192, 70aa12]","667497,5c1bb1,ac2136",False
17149,192a28,"[fde66d, 70aa12, bd59c4]","192a28,70aa12",True
26021,c5f033,"[e051ee, 31299d, bd59c4]","c5f033,c5f033,e051ee",True
24035,8fc9b1,"[3254e9, 524e24, cf13ba]","8fc9b1,667497",False


In [ ]:
# evaluation_set[['first_item','Recommendation','items','Hit']]

## Cosine similarity

In [ ]:
# recommendations system using cosine similarity
def cosine_sim(Customer_Item_matrix):
  item_item_similarity_matrix = pd.DataFrame(cosine_similarity(Customer_Item_matrix.T))
  item_item_similarity_matrix.columns = Customer_Item_matrix.T.index
  item_item_similarity_matrix['Items'] = Customer_Item_matrix.T.index
  item_item_similarity_matrix = item_item_similarity_matrix.set_index('Items')
  return item_item_similarity_matrix

item_item_similarity_matrix = cosine_sim(Customer_Item_matrix)
# Display header of Item to Item similarity matrix. 
# item_item_similarity_matrix.head()

### Evaluation

In [ ]:
# Random test
top_3_similar_items = list(item_item_similarity_matrix.loc['7356a8'].sort_values(ascending=False).iloc[1:4].index)
# top_3_similar_items
recommnd = top_3_similar_items
d = pd.DataFrame(test_set['items'].loc[test_set['items'].str.contains("7356a8")]) 
hit = []
recommnd_list=[]
total = 0
for i in d['items']:
  total +=1
  if recommnd[0] in i:
    hit.append("True")
    recommnd_list.append(recommnd[0])
    continue
  elif recommnd[1] in i:
    hit.append("True")
    recommnd_list.append(recommnd[1])
    continue
  elif recommnd[2] in i:
    hit.append("True")
    recommnd_list.append(recommnd[2])
    continue
  else:
    hit.append('False')
    recommnd_list.append("0")
  
d["Hit"] = hit
d["test"] = recommnd_list
print(total)
print(d["Hit"].value_counts())
print(d['test'].value_counts())
d.head()

841
True     467
False    374
Name: Hit, dtype: int64
363929    376
0         374
dc9f6a     53
8319aa     38
Name: test, dtype: int64


,items,Hit,test
6423,"7356a8,363929",True,363929
14209,"7356a8,7356a8,363929,1bfcc7",True,363929
25489,"dc9f6a,7356a8,363929,8319aa",True,363929
11139,"7356a8,363929,363929,550e0e,b31707",True,363929
19512,"7356a8,c2456b,363929,363929",True,363929


In [ ]:
evaluation_set = test_set.copy()

In [ ]:
# top 3 recommendations 
def recommendations_set(evaluation_set,item_item_similarity_matrix):
  Recommendation =[]
  for i in evaluation_set['first_item']:
    Recommendation.append(list(item_item_similarity_matrix.loc[i].sort_values(ascending=False).iloc[1:4].index))
  evaluation_set["Recommendation"] = Recommendation
  return evaluation_set


evaluation_set = recommendations_set(evaluation_set,item_item_similarity_matrix)
evaluation_set.head()

,tid,date,daypart,channel,items,No_of_items,first_item,Recommendation
6423,8235,2016-03-08,A,3,"7356a8,363929",2,7356a8,"[363929, 8319aa, dc9f6a]"
11685,15147,2016-03-13,B,3,"31299d,c5f033,ea300a",3,31299d,"[e051ee, ea300a, c5f033]"
14209,18375,2016-03-16,A,2,"7356a8,7356a8,363929,1bfcc7",4,7356a8,"[363929, 8319aa, dc9f6a]"
4895,6269,2016-03-06,E,2,"31299d,31299d,31299d,c5f033,c5f033,ea300a,ea300a",7,31299d,"[e051ee, ea300a, c5f033]"
15853,20582,2016-03-17,E,2,"c5f033,c5f033,ea32e3,70aa12,ea300a",5,c5f033,"[e051ee, 31299d, ea300a]"


In [ ]:
# Recommendation system result
result = evaluation(evaluation_set)
result.head()
result[['first_item','Recommendation','items','Hit']]

6309
False    3695
True     2614
Name: Hit, dtype: int64


,first_item,Recommendation,items,Hit
6423,7356a8,"[363929, 8319aa, dc9f6a]","7356a8,363929",True
11685,31299d,"[e051ee, ea300a, c5f033]","31299d,c5f033,ea300a",True
14209,7356a8,"[363929, 8319aa, dc9f6a]","7356a8,7356a8,363929,1bfcc7",True
4895,31299d,"[e051ee, ea300a, c5f033]","31299d,31299d,31299d,c5f033,c5f033,ea300a,ea300a",True
15853,c5f033,"[e051ee, 31299d, ea300a]","c5f033,c5f033,ea32e3,70aa12,ea300a",True
...,...,...,...,...
30326,667497,"[e051ee, 70aa12, ea300a]","667497,5c1bb1,ac2136",False
17149,192a28,"[e051ee, 70aa12, fde66d]","192a28,70aa12",True
26021,c5f033,"[e051ee, 31299d, ea300a]","c5f033,c5f033,e051ee",True
24035,8fc9b1,"[3254e9, 7356a8, 524e24]","8fc9b1,667497",False


In [ ]:
# testing data with "No_of_items" >3
evaluation_set = test_set.copy()
evaluation_set = evaluation_set[evaluation_set["No_of_items"] > 3]
evaluation_set = recommendations_set(evaluation_set,item_item_similarity_matrix)
evaluation_set.head()
result = evaluation(evaluation_set)
result.head()
result[['first_item','Recommendation','items','Hit']]

2656
True     1551
False    1105
Name: Hit, dtype: int64


,first_item,Recommendation,items,Hit
14209,7356a8,"[363929, 8319aa, dc9f6a]","7356a8,7356a8,363929,1bfcc7",True
4895,31299d,"[e051ee, ea300a, c5f033]","31299d,31299d,31299d,c5f033,c5f033,ea300a,ea300a",True
15853,c5f033,"[e051ee, 31299d, ea300a]","c5f033,c5f033,ea32e3,70aa12,ea300a",True
20057,c5f033,"[e051ee, 31299d, ea300a]","c5f033,31299d,c5f033,e051ee,5c1bb1",True
20862,c5f033,"[e051ee, 31299d, ea300a]","c5f033,c5f033,6ca4a6,667497,fde66d,ea300a",True
...,...,...,...,...
21283,e33718,"[230526, dd37cb, 4c85c6]","e33718,368d36,086cbd,9587fa",False
19128,31299d,"[e051ee, ea300a, c5f033]","31299d,31299d,31299d,c5f033",True
2502,c5f033,"[e051ee, 31299d, ea300a]","c5f033,292491,2f4a70,368d36",False
4046,524e24,"[363929, 8319aa, 6050b0]","524e24,1bfcc7,1bfcc7,1bfcc7,1bfcc7,6050b0",True


In [ ]:
# testing data with "No_of_items" >4
evaluation_set = test_set.copy()
evaluation_set = evaluation_set[evaluation_set["No_of_items"] > 4]
evaluation_set = recommendations_set(evaluation_set,item_item_similarity_matrix)
evaluation_set.head()
result = evaluation(evaluation_set)
result.head()
result[['first_item','Recommendation','items','Hit']]

1583
True     1002
False     581
Name: Hit, dtype: int64


,first_item,Recommendation,items,Hit
4895,31299d,"[e051ee, ea300a, c5f033]","31299d,31299d,31299d,c5f033,c5f033,ea300a,ea300a",True
15853,c5f033,"[e051ee, 31299d, ea300a]","c5f033,c5f033,ea32e3,70aa12,ea300a",True
20057,c5f033,"[e051ee, 31299d, ea300a]","c5f033,31299d,c5f033,e051ee,5c1bb1",True
20862,c5f033,"[e051ee, 31299d, ea300a]","c5f033,c5f033,6ca4a6,667497,fde66d,ea300a",True
11139,7356a8,"[363929, 8319aa, dc9f6a]","7356a8,363929,363929,550e0e,b31707",True
...,...,...,...,...
6846,3a9985,"[d910b5, 9fc08c, afe15d]","3a9985,a378da,fb1718,363929,363929",False
22053,7356a8,"[363929, 8319aa, dc9f6a]","7356a8,7356a8,7356a8,363929,363929,ea300a",True
3693,192a28,"[e051ee, 70aa12, fde66d]","192a28,ea300a,696343,b31707,292491,70aa12",True
28265,b65c47,"[e051ee, bf0192, 70aa12]","b65c47,e051ee,6050b0,31299d,31299d",True
